# Analytical Insights Endpoint
## Comprehensive Stock Analysis and Visualization System

This notebook provides a complete analytical insights system that:
1. Accepts user inputs for company analysis
2. Fetches and validates stock data
3. Computes comprehensive financial metrics
4. Generates interactive visualizations
5. Delivers actionable insights

In [2]:
# Run this cell to install required packages
!pip install flask yfinance requests

In [3]:
# Import required libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import json
from flask import Flask, request, jsonify
import re

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
print("Libraries imported successfully!")

Libraries imported successfully!


In [4]:
# Step 1: Data Ingestion and API Setup Functions

def validate_date_format(date_string):
    """Validate date format DD/MM/YYYY"""
    try:
        datetime.strptime(date_string, '%d/%m/%Y')
        return True
    except ValueError:
        return False

def validate_date_range(start_date, end_date):
    """Validate that start date is before end date"""
    start = datetime.strptime(start_date, '%d/%m/%Y')
    end = datetime.strptime(end_date, '%d/%m/%Y')
    return start < end

def get_ticker_from_company_name(company_name):
    """Attempt to resolve company name to ticker symbol"""
    # Common company name to ticker mappings
    company_mappings = {
        'apple': 'AAPL', 'microsoft': 'MSFT', 'google': 'GOOGL', 'alphabet': 'GOOGL',
        'amazon': 'AMZN', 'tesla': 'TSLA', 'meta': 'META', 'facebook': 'META',
        'netflix': 'NFLX', 'nvidia': 'NVDA', 'intel': 'INTC', 'amd': 'AMD',
        'ibm': 'IBM', 'oracle': 'ORCL', 'salesforce': 'CRM', 'adobe': 'ADBE'
    }

    company_lower = company_name.lower().strip()
    if company_lower in company_mappings:
        return company_mappings[company_lower]

    # If direct mapping not found, try as ticker
    return company_name.upper()

def validate_ticker(ticker):
    """Validate if ticker exists in Yahoo Finance"""
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        return 'symbol' in info or 'shortName' in info
    except:
        return False

def fetch_stock_data(ticker, start_date, end_date):
    """Fetch historical stock data from Yahoo Finance"""
    try:
        start = datetime.strptime(start_date, '%d/%m/%Y')
        end = datetime.strptime(end_date, '%d/%m/%Y')

        stock = yf.Ticker(ticker)
        data = stock.history(start=start, end=end)

        if data.empty:
            return None

        return data
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

print("Data ingestion functions defined successfully!")

Data ingestion functions defined successfully!


In [5]:
# Step 2: Data Processing and Analytical Computation Functions

def calculate_moving_averages(data, short_window=20, long_window=50):
    """Calculate Simple and Exponential Moving Averages"""
    data[f'SMA_{short_window}'] = data['Close'].rolling(window=short_window).mean()
    data[f'SMA_{long_window}'] = data['Close'].rolling(window=long_window).mean()
    data[f'EMA_{short_window}'] = data['Close'].ewm(span=short_window).mean()
    data[f'EMA_{long_window}'] = data['Close'].ewm(span=long_window).mean()
    return data

def calculate_returns(data):
    """Calculate daily and cumulative returns"""
    data['Daily_Return'] = data['Close'].pct_change()
    data['Cumulative_Return'] = (1 + data['Daily_Return']).cumprod() - 1
    return data

def calculate_volatility(returns, window=252):
    """Calculate annualized volatility"""
    return returns.std() * np.sqrt(window)

def calculate_sharpe_ratio(returns, risk_free_rate=0.02):
    """Calculate Sharpe ratio"""
    excess_returns = returns.mean() * 252 - risk_free_rate
    volatility = calculate_volatility(returns)
    return excess_returns / volatility if volatility != 0 else 0

def calculate_beta(stock_returns, market_returns):
    """Calculate Beta vs benchmark"""
    aligned_data = pd.concat([stock_returns, market_returns], axis=1).dropna()
    if len(aligned_data) < 2:
        return 0
    covariance = np.cov(aligned_data.iloc[:, 0], aligned_data.iloc[:, 1])[0][1]
    market_variance = np.var(aligned_data.iloc[:, 1])
    return covariance / market_variance if market_variance != 0 else 0

def calculate_rsi(data, window=14):
    """Calculate Relative Strength Index"""
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    data['RSI'] = 100 - (100 / (1 + rs))
    return data

def calculate_macd(data, fast=12, slow=26, signal=9):
    """Calculate MACD indicator"""
    ema_fast = data['Close'].ewm(span=fast).mean()
    ema_slow = data['Close'].ewm(span=slow).mean()
    data['MACD'] = ema_fast - ema_slow
    data['MACD_Signal'] = data['MACD'].ewm(span=signal).mean()
    data['MACD_Histogram'] = data['MACD'] - data['MACD_Signal']
    return data

def calculate_bollinger_bands(data, window=20, num_std=2):
    """Calculate Bollinger Bands"""
    rolling_mean = data['Close'].rolling(window=window).mean()
    rolling_std = data['Close'].rolling(window=window).std()
    data['BB_Upper'] = rolling_mean + (rolling_std * num_std)
    data['BB_Lower'] = rolling_mean - (rolling_std * num_std)
    data['BB_Middle'] = rolling_mean
    return data

def calculate_max_drawdown(data):
    """Calculate maximum drawdown and recovery period"""
    cumulative = (1 + data['Daily_Return']).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max

    max_drawdown = drawdown.min()
    max_dd_date = drawdown.idxmin()

    # Find recovery period
    recovery_date = None
    if max_dd_date in drawdown.index:
        post_dd = drawdown[max_dd_date:]
        recovery_idx = post_dd[post_dd >= -0.01].index  # Within 1% of peak
        if len(recovery_idx) > 0:
            recovery_date = recovery_idx[0]

    return {
        'max_drawdown': max_drawdown,
        'max_drawdown_date': max_dd_date,
        'recovery_date': recovery_date,
        'drawdown_series': drawdown
    }

def detect_anomalies(data, threshold=3):
    """Detect price and volume anomalies using z-score"""
    price_changes = data['Close'].pct_change().abs()
    volume_changes = data['Volume'].pct_change().abs()

    price_z_scores = np.abs((price_changes - price_changes.mean()) / price_changes.std())
    volume_z_scores = np.abs((volume_changes - volume_changes.mean()) / volume_changes.std())

    price_anomalies = data[price_z_scores > threshold].index
    volume_anomalies = data[volume_z_scores > threshold].index

    return {
        'price_anomalies': price_anomalies.tolist(),
        'volume_anomalies': volume_anomalies.tolist()
    }

print("Analytical computation functions defined successfully!")

Analytical computation functions defined successfully!


In [6]:
# Step 3: Interactive User Input Function

def get_company_info_interactive():
    """Interactive function to get user inputs for company analysis"""
    print("=== Company Analysis Input Interface ===")
    print("Please provide the following information:")

    # Get company name
    while True:
        company_name = input("\nEnter Company Name (e.g., Apple, AAPL): ").strip()
        if company_name:
            ticker = get_ticker_from_company_name(company_name)
            print(f"Resolved ticker symbol: {ticker}")

            if validate_ticker(ticker):
                print(f"✓ Ticker {ticker} validated successfully")
                break
            else:
                print(f"✗ Ticker {ticker} not found. Please try again.")
        else:
            print("Please enter a valid company name.")

    # Get start date
    while True:
        start_date = input("\nEnter Start Date (DD/MM/YYYY): ").strip()
        if validate_date_format(start_date):
            print(f"✓ Start date {start_date} is valid")
            break
        else:
            print("✗ Invalid date format. Please use DD/MM/YYYY")

    # Get end date
    while True:
        end_date = input("\nEnter End Date (DD/MM/YYYY): ").strip()
        if validate_date_format(end_date):
            if validate_date_range(start_date, end_date):
                print(f"✓ End date {end_date} is valid")
                break
            else:
                print("✗ End date must be after start date")
        else:
            print("✗ Invalid date format. Please use DD/MM/YYYY")

    # Get benchmark ticker
    while True:
        benchmark = input("\nEnter Benchmark Ticker (e.g., SPY, ^GSPC): ").strip().upper()
        if benchmark:
            if validate_ticker(benchmark):
                print(f"✓ Benchmark {benchmark} validated successfully")
                break
            else:
                print(f"✗ Benchmark {benchmark} not found. Please try again.")
        else:
            print("Please enter a valid benchmark ticker.")

    return {
        'company_name': company_name,
        'ticker': ticker,
        'start_date': start_date,
        'end_date': end_date,
        'benchmark': benchmark
    }

print("Interactive input function defined successfully!")

Interactive input function defined successfully!


In [7]:
# Step 4: Comprehensive Analysis Function

def perform_comprehensive_analysis(company_info):
    """Perform complete analysis on the provided company data"""
    ticker = company_info['ticker']
    benchmark = company_info['benchmark']
    start_date = company_info['start_date']
    end_date = company_info['end_date']

    print(f"\n=== Analyzing {ticker} vs {benchmark} ===")
    print(f"Period: {start_date} to {end_date}")

    # Fetch data
    print("\nFetching stock data...")
    stock_data = fetch_stock_data(ticker, start_date, end_date)
    benchmark_data = fetch_stock_data(benchmark, start_date, end_date)

    if stock_data is None or benchmark_data is None:
        return None

    # Process stock data
    print("Processing analytical metrics...")
    stock_data = calculate_moving_averages(stock_data)
    stock_data = calculate_returns(stock_data)
    stock_data = calculate_rsi(stock_data)
    stock_data = calculate_macd(stock_data)
    stock_data = calculate_bollinger_bands(stock_data)

    # Process benchmark data
    benchmark_data = calculate_returns(benchmark_data)

    # Calculate metrics
    stock_volatility = calculate_volatility(stock_data['Daily_Return'].dropna())
    benchmark_volatility = calculate_volatility(benchmark_data['Daily_Return'].dropna())

    stock_sharpe = calculate_sharpe_ratio(stock_data['Daily_Return'].dropna())
    benchmark_sharpe = calculate_sharpe_ratio(benchmark_data['Daily_Return'].dropna())

    beta = calculate_beta(stock_data['Daily_Return'].dropna(), benchmark_data['Daily_Return'].dropna())

    drawdown_info = calculate_max_drawdown(stock_data.dropna())
    anomalies = detect_anomalies(stock_data.dropna())

    # Calculate performance metrics
    total_return_stock = stock_data['Cumulative_Return'].iloc[-1]
    total_return_benchmark = benchmark_data['Cumulative_Return'].iloc[-1]

    results = {
        'company_info': company_info,
        'stock_data': stock_data,
        'benchmark_data': benchmark_data,
        'metrics': {
            'total_return_stock': total_return_stock,
            'total_return_benchmark': total_return_benchmark,
            'stock_volatility': stock_volatility,
            'benchmark_volatility': benchmark_volatility,
            'stock_sharpe': stock_sharpe,
            'benchmark_sharpe': benchmark_sharpe,
            'beta': beta,
            'max_drawdown': drawdown_info['max_drawdown'],
            'max_drawdown_date': str(drawdown_info['max_drawdown_date']),
            'recovery_date': str(drawdown_info['recovery_date']) if drawdown_info['recovery_date'] else None
        },
        'anomalies': anomalies,
        'drawdown_series': drawdown_info['drawdown_series']
    }

    return results

print("Comprehensive analysis function defined successfully!")

Comprehensive analysis function defined successfully!


In [8]:
# Step 4: Visualization Functions

def create_price_trend_chart(results):
    """Create interactive price trend chart with moving averages"""
    stock_data = results['stock_data']
    ticker = results['company_info']['ticker']

    fig = go.Figure()

    # Add price line
    fig.add_trace(go.Scatter(
        x=stock_data.index,
        y=stock_data['Close'],
        mode='lines',
        name=f'{ticker} Price',
        line=dict(color='blue', width=2)
    ))

    # Add moving averages
    fig.add_trace(go.Scatter(
        x=stock_data.index,
        y=stock_data['SMA_20'],
        mode='lines',
        name='SMA 20',
        line=dict(color='orange', width=1)
    ))

    fig.add_trace(go.Scatter(
        x=stock_data.index,
        y=stock_data['SMA_50'],
        mode='lines',
        name='SMA 50',
        line=dict(color='red', width=1)
    ))

    # Add Bollinger Bands
    fig.add_trace(go.Scatter(
        x=stock_data.index,
        y=stock_data['BB_Upper'],
        mode='lines',
        name='BB Upper',
        line=dict(color='gray', width=1, dash='dash'),
        showlegend=False
    ))

    fig.add_trace(go.Scatter(
        x=stock_data.index,
        y=stock_data['BB_Lower'],
        mode='lines',
        name='BB Lower',
        line=dict(color='gray', width=1, dash='dash'),
        fill='tonexty',
        fillcolor='rgba(128,128,128,0.1)'
    ))

    fig.update_layout(
        title=f'{ticker} Price Trend with Technical Indicators',
        xaxis_title='Date',
        yaxis_title='Price ($)',
        hovermode='x unified'
    )

    return fig

def create_returns_comparison_chart(results):
    """Create returns comparison chart"""
    stock_data = results['stock_data']
    benchmark_data = results['benchmark_data']
    ticker = results['company_info']['ticker']
    benchmark = results['company_info']['benchmark']

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=stock_data.index,
        y=stock_data['Cumulative_Return'] * 100,
        mode='lines',
        name=f'{ticker} Returns',
        line=dict(color='blue', width=2)
    ))

    fig.add_trace(go.Scatter(
        x=benchmark_data.index,
        y=benchmark_data['Cumulative_Return'] * 100,
        mode='lines',
        name=f'{benchmark} Returns',
        line=dict(color='red', width=2)
    ))

    fig.update_layout(
        title=f'Cumulative Returns: {ticker} vs {benchmark}',
        xaxis_title='Date',
        yaxis_title='Cumulative Return (%)',
        hovermode='x unified'
    )

    return fig

def create_technical_indicators_chart(results):
    """Create technical indicators chart (RSI, MACD)"""
    stock_data = results['stock_data']
    ticker = results['company_info']['ticker']

    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=['RSI', 'MACD'],
        vertical_spacing=0.1
    )

    # RSI
    fig.add_trace(go.Scatter(
        x=stock_data.index,
        y=stock_data['RSI'],
        mode='lines',
        name='RSI',
        line=dict(color='purple')
    ), row=1, col=1)

    # RSI overbought/oversold lines
    fig.add_hline(y=70, line_dash="dash", line_color="red", row=1, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", row=1, col=1)

    # MACD
    fig.add_trace(go.Scatter(
        x=stock_data.index,
        y=stock_data['MACD'],
        mode='lines',
        name='MACD',
        line=dict(color='blue')
    ), row=2, col=1)

    fig.add_trace(go.Scatter(
        x=stock_data.index,
        y=stock_data['MACD_Signal'],
        mode='lines',
        name='Signal',
        line=dict(color='red')
    ), row=2, col=1)

    fig.add_trace(go.Bar(
        x=stock_data.index,
        y=stock_data['MACD_Histogram'],
        name='Histogram',
        marker_color='gray',
        opacity=0.6
    ), row=2, col=1)

    fig.update_layout(
        title=f'{ticker} Technical Indicators',
        height=600
    )

    return fig

def create_risk_metrics_chart(results):
    """Create risk metrics visualization"""
    metrics = results['metrics']
    ticker = results['company_info']['ticker']
    benchmark = results['company_info']['benchmark']

    # Create comparison chart
    categories = ['Total Return (%)', 'Volatility (%)', 'Sharpe Ratio']
    stock_values = [
        metrics['total_return_stock'] * 100,
        metrics['stock_volatility'] * 100,
        metrics['stock_sharpe']
    ]
    benchmark_values = [
        metrics['total_return_benchmark'] * 100,
        metrics['benchmark_volatility'] * 100,
        metrics['benchmark_sharpe']
    ]

    fig = go.Figure(data=[
        go.Bar(name=ticker, x=categories, y=stock_values, marker_color='blue'),
        go.Bar(name=benchmark, x=categories, y=benchmark_values, marker_color='red')
    ])

    fig.update_layout(
        title=f'Risk-Return Metrics: {ticker} vs {benchmark}',
        barmode='group',
        yaxis_title='Value'
    )

    return fig

def create_drawdown_chart(results):
    """Create drawdown visualization"""
    drawdown_series = results['drawdown_series']
    ticker = results['company_info']['ticker']

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=drawdown_series.index,
        y=drawdown_series * 100,
        mode='lines',
        name='Drawdown',
        fill='tozeroy',
        fillcolor='rgba(255,0,0,0.3)',
        line=dict(color='red')
    ))

    fig.update_layout(
        title=f'{ticker} Drawdown Analysis',
        xaxis_title='Date',
        yaxis_title='Drawdown (%)',
        hovermode='x unified'
    )

    return fig

print("Visualization functions defined successfully!")

Visualization functions defined successfully!


In [9]:
# Generate Insights and Summary Report

def generate_insights_report(results):
    """Generate comprehensive text-based insights report"""
    metrics = results['metrics']
    ticker = results['company_info']['ticker']
    benchmark = results['company_info']['benchmark']
    company_name = results['company_info']['company_name']

    report = f"""
    ═══════════════════════════════════════════════════════════════
                    ANALYTICAL INSIGHTS REPORT
    ═══════════════════════════════════════════════════════════════

    Company: {company_name} ({ticker})
    Benchmark: {benchmark}
    Analysis Period: {results['company_info']['start_date']} to {results['company_info']['end_date']}

    ───────────────────────────────────────────────────────────────
    📊 PERFORMANCE SUMMARY
    ───────────────────────────────────────────────────────────────

    Total Return:
    • {ticker}: {metrics['total_return_stock']:.2%}
    • {benchmark}: {metrics['total_return_benchmark']:.2%}
    • Outperformance: {(metrics['total_return_stock'] - metrics['total_return_benchmark']):.2%}

    ───────────────────────────────────────────────────────────────
    ⚠️  RISK ANALYSIS
    ───────────────────────────────────────────────────────────────

    Volatility (Annualized):
    • {ticker}: {metrics['stock_volatility']:.2%}
    • {benchmark}: {metrics['benchmark_volatility']:.2%}

    Risk-Adjusted Returns:
    • {ticker} Sharpe Ratio: {metrics['stock_sharpe']:.3f}
    • {benchmark} Sharpe Ratio: {metrics['benchmark_sharpe']:.3f}

    Market Sensitivity:
    • Beta vs {benchmark}: {metrics['beta']:.3f}

    Maximum Drawdown:
    • Worst Loss: {metrics['max_drawdown']:.2%}
    • Occurred on: {metrics['max_drawdown_date']}
    • Recovery Date: {metrics['recovery_date'] if metrics['recovery_date'] != 'None' else 'Not yet recovered'}

    ───────────────────────────────────────────────────────────────
    🎯 KEY INSIGHTS & RECOMMENDATIONS
    ───────────────────────────────────────────────────────────────
    """

    # Performance insights
    if metrics['total_return_stock'] > metrics['total_return_benchmark']:
        report += f"\n    ✅ OUTPERFORMANCE: {ticker} outperformed {benchmark} by {(metrics['total_return_stock'] - metrics['total_return_benchmark']):.2%}"
    else:
        report += f"\n    ❌ UNDERPERFORMANCE: {ticker} underperformed {benchmark} by {(metrics['total_return_benchmark'] - metrics['total_return_stock']):.2%}"

    # Risk insights
    if metrics['stock_volatility'] > metrics['benchmark_volatility']:
        report += f"\n    ⚠️  HIGH RISK: {ticker} is {(metrics['stock_volatility']/metrics['benchmark_volatility'] - 1):.1%} more volatile than {benchmark}"
    else:
        report += f"\n    ✅ LOWER RISK: {ticker} is {(1 - metrics['stock_volatility']/metrics['benchmark_volatility']):.1%} less volatile than {benchmark}"

    # Sharpe ratio insights
    if metrics['stock_sharpe'] > metrics['benchmark_sharpe']:
        report += f"\n    ✅ BETTER RISK-ADJUSTED RETURNS: {ticker} has superior risk-adjusted performance"
    else:
        report += f"\n    ❌ POOR RISK-ADJUSTED RETURNS: {benchmark} offers better risk-adjusted returns"

    # Beta insights
    if metrics['beta'] > 1.2:
        report += f"\n    📈 HIGH BETA STOCK: {ticker} is highly sensitive to market movements (Beta: {metrics['beta']:.2f})"
    elif metrics['beta'] < 0.8:
        report += f"\n    📉 LOW BETA STOCK: {ticker} is less sensitive to market movements (Beta: {metrics['beta']:.2f})"
    else:
        report += f"\n    ⚖️  MODERATE BETA: {ticker} moves roughly in line with the market (Beta: {metrics['beta']:.2f})"

    # Drawdown insights
    if abs(metrics['max_drawdown']) > 0.2:
        report += f"\n    🚨 HIGH DRAWDOWN RISK: Maximum loss of {metrics['max_drawdown']:.1%} indicates high downside risk"
    elif abs(metrics['max_drawdown']) > 0.1:
        report += f"\n    ⚠️  MODERATE DRAWDOWN: Maximum loss of {metrics['max_drawdown']:.1%} shows moderate downside risk"
    else:
        report += f"\n    ✅ LOW DRAWDOWN: Maximum loss of {metrics['max_drawdown']:.1%} indicates good downside protection"

    report += "\n\n    ═══════════════════════════════════════════════════════════════\n"

    return report

def display_all_visualizations(results):
    """Display all charts and insights"""
    print("\n🎨 Generating visualizations...")

    # Generate insights report
    insights = generate_insights_report(results)
    print(insights)

    # Create and display charts
    charts = {
        'price_trend': create_price_trend_chart(results),
        'returns_comparison': create_returns_comparison_chart(results),
        'technical_indicators': create_technical_indicators_chart(results),
        'risk_metrics': create_risk_metrics_chart(results),
        'drawdown': create_drawdown_chart(results)
    }

    for chart_name, fig in charts.items():
        fig.show()

    return charts, insights

print("Insights and reporting functions defined successfully!")

Insights and reporting functions defined successfully!


In [10]:
# Main Execution Cell
# Run this cell to start the interactive company lookup
# This will ask user to user inputs: Company Name, Start Date, End Date, and Benchmark Ticker.

def main_analysis_workflow():
    """Main workflow for comprehensive stock analysis"""
    try:
        # Step 1: Get user inputs
        print("🚀 Starting Analytical Insights Endpoint...")
        company_info = get_company_info_interactive()

        # Step 2: Perform comprehensive analysis
        results = perform_comprehensive_analysis(company_info)

        if results is None:
            print("❌ Analysis failed. Please check your inputs and try again.")
            return None

        # Step 3: Generate visualizations and insights
        charts, insights = display_all_visualizations(results)

        print("\n✅ Analysis completed successfully!")
        print("📊 All charts have been generated and displayed above.")
        print("📋 Detailed insights report has been provided.")

        return {
            'results': results,
            'charts': charts,
            'insights': insights
        }

    except Exception as e:
        print(f"❌ Error during analysis: {str(e)}")
        return None

# Execute the main workflow
company_info = get_company_info_interactive()

=== Company Analysis Input Interface ===
Please provide the following information:

Enter Company Name (e.g., Apple, AAPL): google
Resolved ticker symbol: GOOGL
✓ Ticker GOOGL validated successfully

Enter Start Date (DD/MM/YYYY): 14/01/2025
✓ Start date 14/01/2025 is valid

Enter End Date (DD/MM/YYYY): 14/05/2025
✓ End date 14/05/2025 is valid

Enter Benchmark Ticker (e.g., SPY, ^GSPC): SPY
✓ Benchmark SPY validated successfully


In [11]:
# Execute Complete Analysis
# Run this cell after providing inputs in the previous cell

if 'company_info' in locals():
    print("\n🔄 Running comprehensive analysis...")
    analysis_results = main_analysis_workflow()

    if analysis_results:
        print("\n🎉 Analysis completed! Check the visualizations and insights above.")
    else:
        print("\n❌ Analysis failed. Please check the error messages above.")
else:
    print("⚠️  Please run the previous cell first to provide company information.")


🔄 Running comprehensive analysis...
🚀 Starting Analytical Insights Endpoint...
=== Company Analysis Input Interface ===
Please provide the following information:

Enter Company Name (e.g., Apple, AAPL): Apple
Resolved ticker symbol: AAPL
✓ Ticker AAPL validated successfully

Enter Start Date (DD/MM/YYYY): 14/01/2025
✓ Start date 14/01/2025 is valid

Enter End Date (DD/MM/YYYY): 14/05/2025
✓ End date 14/05/2025 is valid

Enter Benchmark Ticker (e.g., SPY, ^GSPC): SPY
✓ Benchmark SPY validated successfully

=== Analyzing AAPL vs SPY ===
Period: 14/01/2025 to 14/05/2025

Fetching stock data...
Processing analytical metrics...

🎨 Generating visualizations...

    ═══════════════════════════════════════════════════════════════
                    ANALYTICAL INSIGHTS REPORT
    ═══════════════════════════════════════════════════════════════
    
    Company: Apple (AAPL)
    Benchmark: SPY
    Analysis Period: 14/01/2025 to 14/05/2025
    
    ───────────────────────────────────────────────


✅ Analysis completed successfully!
📊 All charts have been generated and displayed above.
📋 Detailed insights report has been provided.

🎉 Analysis completed! Check the visualizations and insights above.
